# SPK Derivatives: Demo with Context
Run-through with model context, sanity checks, pricing, Greeks, and location comparison.

## Model framing
- Underlying: daily solar energy price (kWh → $) from NASA irradiance.
- Dynamics: risk-neutral GBM on price S with volatility from log returns (optional cap for stability).
- Payoff: European call on energy (max(S_T - K, 0)), ATM by default.
- Methods: binomial tree and Monte Carlo; Greeks via finite differences.
- Scope: European-style; American/barrier on roadmap.

In [ ]:

# Setup: import the package. If not installed, try local path, then pip-install.
import sys, subprocess
from pathlib import Path

try:
    import spk_derivatives
except ImportError:
    root = Path().resolve()
    local_pkg = root / "energy_derivatives"
    if (local_pkg / "spk_derivatives").exists():
        sys.path.insert(0, str(local_pkg))
        import spk_derivatives
    else:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "git+https://github.com/Spectating101/spk-derivatives.git"])
        import spk_derivatives

from spk_derivatives import (
    load_solar_parameters,
    BinomialTree,
    MonteCarloSimulator,
    calculate_greeks,
)
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')


## 1) Load solar parameters (Taiwan)
- Log-return volatility, capped at 200% for stability.
- Cached NASA data keeps it offline-friendly.

In [ ]:

params_cap = load_solar_parameters(
    lat=24.99, lon=121.30,
    volatility_cap=2.0,
    volatility_method='log',
    cache=True,
)
summary = {k: params_cap[k] for k in ['S0','sigma','T','r','volatility_method','volatility_cap','location']}
summary


## 2) Sanity check: raw vs capped volatility
Shows transparency: raw log-return sigma vs capped sigma.

In [ ]:

params_raw = load_solar_parameters(
    lat=24.99, lon=121.30,
    volatility_cap=None,
    volatility_method='log',
    cache=True,
)
vols = {
    'sigma_raw': params_raw['sigma'],
    'sigma_capped': params_cap['sigma'],
    'raw_over_cap': params_raw['sigma'] / params_cap['sigma'] if params_cap['sigma'] else None,
}
vols


## 3) Price with binomial and Monte Carlo (95% CI)

In [ ]:

core = {k: params_cap[k] for k in ('S0','K','T','r','sigma')}

# Binomial
binomial_price = BinomialTree(**core, N=400, payoff_type='call').price()

# Monte Carlo
mc = MonteCarloSimulator(**core, num_simulations=20000, seed=7, payoff_type='call')
mc_price, mc_low, mc_high = mc.confidence_interval(confidence=0.95)

pricing = {
    'binomial_price': round(binomial_price, 6),
    'mc_price': round(mc_price, 6),
    'mc_ci_low': round(mc_low, 6),
    'mc_ci_high': round(mc_high, 6),
}
pricing


## 4) Convergence mini-check (binomial steps) vs MC reference

In [ ]:

steps_list = [50, 100, 200, 400]
rows = []
for n in steps_list:
    price_n = BinomialTree(**core, N=n, payoff_type='call').price()
    rows.append({'N': n, 'Binomial': price_n})
df_conv = pd.DataFrame(rows)
df_conv['MC_ref'] = mc_price
print(df_conv)


## 5) Greeks (binomial)

In [ ]:

greeks_df = calculate_greeks(**core, pricing_method='binomial', N=200)
greeks_df


## 6) Multi-location comparison (Taiwan, Arizona, Spain)

In [ ]:

locations = [
    {'name': 'Taiwan', 'lat': 24.99, 'lon': 121.30},
    {'name': 'Arizona', 'lat': 33.45, 'lon': -112.07},
    {'name': 'Spain', 'lat': 40.42, 'lon': -3.70},
]
rows = []
for loc in locations:
    p = load_solar_parameters(lat=loc['lat'], lon=loc['lon'], volatility_cap=2.0, volatility_method='log', cache=True)
    core_loc = {k: p[k] for k in ('S0','K','T','r','sigma')}
    price = BinomialTree(**core_loc, N=400, payoff_type='call').price()
    rows.append({'Location': loc['name'], 'Price': round(price, 6), 'S0': p['S0'], 'Sigma': p['sigma']})

df_loc = pd.DataFrame(rows)
df_loc


## 7) (Optional) Bar chart of prices by location

In [ ]:

ax = df_loc.plot(kind='bar', x='Location', y='Price', legend=False, color='#4c72b0')
ax.set_ylabel('Option Price ($)')
ax.set_title('Energy Option Price by Location')
plt.show()


---
That’s the full story: assumptions → data → pricing → Greeks → location comparison. Use this notebook for a more substantive demo while staying concise.